# Industrial Pump Predictive Maintenance using RNN

**Course:** 62FIT4ATI - Artificial Intelligence

**Topic 2:** Recurrent Neural Network for Predictive Maintenance

This notebook is **fully self-contained** - no external .py files required.

---

In [ ]:
# Setup: Install dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q imbalanced-learn
else:
    print('Running locally')

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 60)
print('Libraries imported!')

In [ ]:
# ============================================================
# HELPER FUNCTIONS (Self-contained - no external imports)
# ============================================================

def get_feature_columns():
    return [f'sensor_{i:02d}' for i in range(52)]

def get_target_column():
    return 'machine_status'

def load_data(filepath):
    df = pd.read_csv(filepath)
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

def get_dynamic_colors(n):
    colors = ['#2ecc71', '#f39c12', '#e74c3c', '#3498db', '#9b59b6']
    return colors[:n]

print('Helper functions defined!')

## Section 1: Load and Inspect Data

In [ ]:
# Load data - UPDATE PATH FOR COLAB
DATA_PATH = 'sensor.csv'  # Change to your path

df = load_data(DATA_PATH)
feature_cols = get_feature_columns()
target_col = get_target_column()

print(f'Dataset Shape: {df.shape}')
print(f'Total samples: {len(df):,}')
df.head()

In [ ]:
# Class distribution - HANDLES ANY NUMBER OF CLASSES
class_counts = df[target_col].value_counts()
class_pct = df[target_col].value_counts(normalize=True) * 100
n_classes = len(class_counts)

print('Class Distribution:')
print('=' * 50)
for cls in class_counts.index:
    print(f'{cls:12s}: {class_counts[cls]:>10,} ({class_pct[cls]:>6.3f}%)')
print('=' * 50)

In [ ]:
# Visualize - DYNAMIC for any number of classes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = get_dynamic_colors(n_classes)

# Bar chart
ax1 = axes[0]
bars = ax1.bar(class_counts.index, class_counts.values, color=colors[:len(class_counts)])
ax1.set_xlabel('Machine Status')
ax1.set_ylabel('Count')
ax1.set_title('Class Distribution')
if class_counts.max() / class_counts.min() > 10:
    ax1.set_yscale('log')
for bar, count in zip(bars, class_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{count:,}', ha='center', va='bottom')

# Pie chart - DYNAMIC explode
ax2 = axes[1]
explode = [0.02 * i for i in range(n_classes)]
ax2.pie(class_counts.values, labels=class_counts.index, autopct='%1.2f%%',
        colors=colors[:len(class_counts)], explode=explode)
ax2.set_title('Class Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Missing values
missing = df[feature_cols].isnull().sum()
print(f'Total missing values: {missing.sum():,}')
if missing.sum() > 0:
    print(missing[missing > 0])

## Section 2: Data Preprocessing

In [ ]:
# Handle missing values
df[feature_cols] = df[feature_cols].ffill().bfill()

# Fill any remaining NaN with column median (for columns that were all NaN)
for col in feature_cols:
    if df[col].isnull().any():
        median_val = df[col].median()
        if pd.isna(median_val):
            df[col] = 0  # If entire column is NaN, fill with 0
        else:
            df[col] = df[col].fillna(median_val)

# Check for constant columns (will cause NaN after scaling)
constant_cols = [col for col in feature_cols if df[col].nunique() <= 1]
if constant_cols:
    print(f'WARNING: Found {len(constant_cols)} constant columns: {constant_cols[:5]}...')
    # Add small noise to constant columns to avoid division by zero
    for col in constant_cols:
        df[col] = df[col] + np.random.normal(0, 1e-6, len(df))

print(f'Missing after fill: {df[feature_cols].isnull().sum().sum()}')
print(f'Inf values: {np.isinf(df[feature_cols].values).sum()}')

In [ ]:
# Encode labels - DYNAMIC based on actual classes in data
actual_classes = df[target_col].unique().tolist()
print(f'Classes in data: {actual_classes}')

label_encoder = LabelEncoder()
label_encoder.fit(actual_classes)
y_encoded = label_encoder.transform(df[target_col])

class_names = list(label_encoder.classes_)
n_classes = len(class_names)
print(f'Encoded classes: {class_names}')
print(f'Number of classes: {n_classes}')

In [ ]:
# Compute class weights - DYNAMIC
class_weights_arr = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weights = dict(enumerate(class_weights_arr))

print('Class Weights:')
for idx, name in enumerate(class_names):
    print(f'  {name}: {class_weights[idx]:.4f}')

In [ ]:
# Prepare features
X = df[feature_cols].values
y = y_encoded

# Train/val/test split - HANDLES ANY NUMBER OF CLASSES (even 1)
# Only use stratify if we have enough samples per class
min_class_count = pd.Series(y).value_counts().min()
use_stratify = min_class_count >= 2 and n_classes > 1

if use_stratify:
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)
else:
    # No stratify for single class or very few samples
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)
    print(f'Note: Stratified split disabled (n_classes={n_classes}, min_samples={min_class_count})')

print(f'Train: {len(X_train):,}, Val: {len(X_val):,}, Test: {len(X_test):,}')

In [ ]:
# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# CRITICAL: Check for NaN/Inf after scaling
def check_and_fix_data(data, name):
    nan_count = np.isnan(data).sum()
    inf_count = np.isinf(data).sum()
    if nan_count > 0 or inf_count > 0:
        print(f'WARNING: {name} has {nan_count} NaN and {inf_count} Inf values - fixing...')
        data = np.nan_to_num(data, nan=0.0, posinf=3.0, neginf=-3.0)
    return data

X_train_scaled = check_and_fix_data(X_train_scaled, 'X_train_scaled')
X_val_scaled = check_and_fix_data(X_val_scaled, 'X_val_scaled')
X_test_scaled = check_and_fix_data(X_test_scaled, 'X_test_scaled')

# Clip extreme values to prevent overflow during training
X_train_scaled = np.clip(X_train_scaled, -10, 10)
X_val_scaled = np.clip(X_val_scaled, -10, 10)
X_test_scaled = np.clip(X_test_scaled, -10, 10)

print('Features normalized!')
print(f'X_train_scaled - min: {X_train_scaled.min():.2f}, max: {X_train_scaled.max():.2f}')

In [ ]:
# Create sequences for RNN - MEMORY OPTIMIZED for Colab
SEQ_LENGTH = 60

def create_sequences(X, y, seq_length):
    """Memory-efficient sequence creation."""
    n_samples = len(X) - seq_length + 1
    n_features = X.shape[1]
    
    # Pre-allocate array (more memory efficient than list append)
    X_seq = np.zeros((n_samples, seq_length, n_features), dtype=np.float32)
    y_seq = np.zeros(n_samples, dtype=y.dtype)
    
    for i in range(n_samples):
        X_seq[i] = X[i:i + seq_length]
        y_seq[i] = y[i + seq_length - 1]
    
    return X_seq, y_seq

import gc
print('Creating sequences (may take a moment)...')

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, SEQ_LENGTH)
gc.collect()  # Free memory
print(f'  Train done: {X_train_seq.shape}')

X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val, SEQ_LENGTH)
gc.collect()
print(f'  Val done: {X_val_seq.shape}')

X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, SEQ_LENGTH)
gc.collect()
print(f'  Test done: {X_test_seq.shape}')

# Clean up original arrays to free memory
del X_train_scaled, X_val_scaled, X_test_scaled
gc.collect()

print(f'Memory used: ~{(X_train_seq.nbytes + X_val_seq.nbytes + X_test_seq.nbytes) / 1e9:.2f} GB')

In [ ]:
# One-hot encode labels
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train_seq, num_classes=n_classes)
y_val_cat = to_categorical(y_val_seq, num_classes=n_classes)
y_test_cat = to_categorical(y_test_seq, num_classes=n_classes)

# Convert to float32 for compatibility
X_train_seq = X_train_seq.astype(np.float32)
X_val_seq = X_val_seq.astype(np.float32)
X_test_seq = X_test_seq.astype(np.float32)
y_train_cat = y_train_cat.astype(np.float32)
y_val_cat = y_val_cat.astype(np.float32)
y_test_cat = y_test_cat.astype(np.float32)

print(f'One-hot shapes: {y_train_cat.shape}')
print(f'Data type: X={X_train_seq.dtype}, y={y_train_cat.dtype}')

## Section 3: Build LSTM Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print(f'TensorFlow: {tf.__version__}')
print(f'GPU: {len(tf.config.list_physical_devices("GPU")) > 0}')

In [ ]:
# Use standard categorical crossentropy - more stable for extreme imbalance
# Class weights will handle the imbalance instead of focal loss
print('Using categorical crossentropy with class weights (more stable than focal loss)')

In [ ]:
# VALIDATION: Check data before training
print('=== DATA VALIDATION ===')
print(f'X_train_seq - NaN: {np.isnan(X_train_seq).sum()}, Inf: {np.isinf(X_train_seq).sum()}')
print(f'y_train_cat - NaN: {np.isnan(y_train_cat).sum()}, Inf: {np.isinf(y_train_cat).sum()}')
print(f'X_train_seq range: [{X_train_seq.min():.2f}, {X_train_seq.max():.2f}]')

# Build model - DYNAMIC n_classes
n_features = len(feature_cols)

model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(SEQ_LENGTH, n_features)),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(n_classes, activation='softmax')  # DYNAMIC
])

# Cap class weights more aggressively to prevent NaN
MAX_WEIGHT = 10.0  # Reduced from 50 to be safer
class_weights_capped = {k: min(v, MAX_WEIGHT) for k, v in class_weights.items()}
print(f'Original weights: {class_weights}')
print(f'Capped weights (max={MAX_WEIGHT}): {class_weights_capped}')

# Use lower learning rate for stability
model.compile(
    optimizer=Adam(learning_rate=0.0005, clipnorm=1.0),  # Reduced from 0.001
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Callbacks
import os
os.makedirs('models', exist_ok=True)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    ModelCheckpoint('models/best_model.keras', monitor='val_loss', save_best_only=True)
]

print('Callbacks ready!')

In [ ]:
# Train model with CAPPED weights
history = model.fit(
    X_train_seq, y_train_cat,
    validation_data=(X_val_seq, y_val_cat),
    epochs=100,
    batch_size=64,
    class_weight=class_weights_capped,  # Use capped weights!
    callbacks=callbacks,
    verbose=1
)

print('Training complete!')

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(history.history['loss'], label='Train')
axes[0].plot(history.history['val_loss'], label='Val')
axes[0].set_title('Loss')
axes[0].legend()

axes[1].plot(history.history['accuracy'], label='Train')
axes[1].plot(history.history['val_accuracy'], label='Val')
axes[1].set_title('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

## Section 4: Evaluation

In [ ]:
# Predictions
y_pred_proba = model.predict(X_test_seq)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = y_test_seq

print(f'Predictions: {len(y_pred)}')

In [ ]:
# Classification report - DYNAMIC class names
print('Classification Report:')
print('=' * 60)
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

In [ ]:
# Confusion matrix - DYNAMIC
cm = confusion_matrix(y_true, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Normalized confusion matrix
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Normalized Confusion Matrix')
plt.tight_layout()
plt.show()

## Section 5: Inference

In [ ]:
# Save model artifacts
import pickle

model.save('models/final_model.keras')

with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print('Model and artifacts saved!')

In [ ]:
# Inference function
def predict_status(sensor_data, model, scaler, label_encoder, seq_length=60):
    """
    Predict machine status from sensor data.
    sensor_data: array of shape (seq_length, n_features) or (n_samples, n_features)
    """
    if len(sensor_data) < seq_length:
        raise ValueError(f'Need at least {seq_length} samples')
    
    # Take last seq_length samples
    data = sensor_data[-seq_length:]
    
    # Scale
    data_scaled = scaler.transform(data)
    
    # Reshape for model
    data_seq = data_scaled.reshape(1, seq_length, -1)
    
    # Predict
    proba = model.predict(data_seq, verbose=0)[0]
    pred_idx = np.argmax(proba)
    pred_label = label_encoder.inverse_transform([pred_idx])[0]
    
    return {
        'prediction': pred_label,
        'confidence': float(proba[pred_idx]),
        'probabilities': {label_encoder.classes_[i]: float(proba[i]) for i in range(len(proba))}
    }

print('Inference function ready!')

In [ ]:
# Test inference
sample_idx = 1000
sample_data = df[feature_cols].iloc[sample_idx:sample_idx + SEQ_LENGTH].values
actual = df[target_col].iloc[sample_idx + SEQ_LENGTH - 1]

result = predict_status(sample_data, model, scaler, label_encoder, SEQ_LENGTH)

print(f'Actual: {actual}')
print(f'Predicted: {result["prediction"]}')
print(f'Confidence: {result["confidence"]:.2%}')
print(f'Probabilities: {result["probabilities"]}')

## Section 6: Conclusion

This notebook demonstrated:
1. Loading and exploring sensor data
2. Handling class imbalance with class weights and focal loss
3. Building an LSTM model for time-series classification
4. Evaluating model performance
5. Creating an inference pipeline

The model dynamically handles any number of classes present in the data.